In [1]:
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score
from tqdm import tqdm

from utils.datasets import get_datasets
from utils.models import get_existing_models, get_dataset_benchmark_models

In [2]:
DATASET_SAMPLE_SIZE = 10

datasets = get_datasets()
datasets_without_center = get_datasets()

for dataset in datasets:
    if len(dataset.dataframe) <= DATASET_SAMPLE_SIZE:
        continue

    dataset.dataframe = dataset.dataframe.iloc[
                        ::len(dataset.dataframe) // DATASET_SAMPLE_SIZE
                        ].head(DATASET_SAMPLE_SIZE)

for dataset in datasets_without_center:
    dataset.dataframe = dataset.dataframe[dataset.dataframe["leaning"] != "center"]

    if len(dataset.dataframe) <= DATASET_SAMPLE_SIZE:
        continue

    dataset.dataframe = dataset.dataframe.iloc[
                        ::len(dataset.dataframe) // DATASET_SAMPLE_SIZE
                        ].head(DATASET_SAMPLE_SIZE)

In [3]:
TRUNCATE_TOKENS = True
# Lambda, so that the generator can be reused.
GET_MODELS = lambda: get_dataset_benchmark_models()

accuracy_results = []

for model_index, model in enumerate(GET_MODELS()):
    print(f"evaluating {model.name} on:")
    accuracy_results.append([])
    for dataset_index in range(len(datasets)):
        dataset = datasets[dataset_index] if model.supports_center_leaning else datasets_without_center[dataset_index]
        print(f"  {dataset.name}")

        predictions = []
        for body_index, body in enumerate(tqdm(dataset.dataframe["body"])):
            try:
                predictions.append(model.predict(body, TRUNCATE_TOKENS))
            except RuntimeError:
                if TRUNCATE_TOKENS:
                    raise
                predictions.append(None)

        valid_indices = [i for i, prediction in enumerate(predictions) if prediction is not None]
        predictions = list(map(lambda prediction: prediction.value, [predictions[i] for i in valid_indices]))
        accuracy = accuracy_score(
            dataset.dataframe["leaning"].iloc[valid_indices].tolist(),
            predictions
        ) if len(predictions) > 0 else 0
        accuracy_results[-1].append(
            f"{len(valid_indices) * accuracy:.0f}/{len(valid_indices)} ({np.round(accuracy * 100, 2)} %)"
        )

evaluating dataset_benchmark/roberta-base/article_bias_prediction on:
  commoncrawl_news_articles


100%|██████████| 10/10 [00:00<00:00, 23.53it/s]


  article_bias_prediction


100%|██████████| 10/10 [00:00<00:00, 31.96it/s]


  qbias


...

100%|██████████| 10/10 [00:00<00:00, 36.73it/s]


  webis_bias_flipper_18


100%|██████████| 10/10 [00:00<00:00, 30.88it/s]


  gpt4_political_bias


100%|██████████| 10/10 [00:00<00:00, 159.01it/s]


In [4]:
results_df = pd.DataFrame(
    accuracy_results,
    index=list(map(lambda model: model.name, GET_MODELS())),
    columns=list(map(lambda dataset: dataset.name, datasets)),
)

results_df

,commoncrawl_news_articles,article_bias_prediction,qbias,polistance_issue_tweets,webis_news_bias_20,webis_bias_flipper_18,gpt4_political_bias
dataset_benchmark/roberta-base/article_bias_prediction,3/10 (30.0 %),6/10 (60.0 %),2/10 (20.0 %),0/10 (0.0 %),2/10 (20.0 %),2/10 (20.0 %),5/10 (50.0 %)
dataset_benchmark/roberta-base/commoncrawl_news_articles,6/10 (60.0 %),2/10 (20.0 %),6/10 (60.0 %),4/10 (40.0 %),4/10 (40.0 %),3/10 (30.0 %),1/10 (10.0 %)
dataset_benchmark/roberta-base/gpt4_political_bias,3/10 (30.0 %),7/10 (70.0 %),2/10 (20.0 %),0/10 (0.0 %),2/10 (20.0 %),2/10 (20.0 %),5/10 (50.0 %)
dataset_benchmark/roberta-base/polistance_issue_tweets,4/10 (40.0 %),4/10 (40.0 %),5/10 (50.0 %),6/10 (60.0 %),5/10 (50.0 %),6/10 (60.0 %),6/10 (60.0 %)
dataset_benchmark/roberta-base/qbias,3/10 (30.0 %),7/10 (70.0 %),2/10 (20.0 %),0/10 (0.0 %),2/10 (20.0 %),2/10 (20.0 %),5/10 (50.0 %)
dataset_benchmark/roberta-base/webis_bias_flipper_18,3/10 (30.0 %),7/10 (70.0 %),2/10 (20.0 %),0/10 (0.0 %),2/10 (20.0 %),2/10 (20.0 %),5/10 (50.0 %)
dataset_benchmark/roberta-base/webis_news_bias_20,3/10 (30.0 %),7/10 (70.0 %),2/10 (20.0 %),0/10 (0.0 %),2/10 (20.0 %),2/10 (20.0 %),5/10 (50.0 %)
dataset_benchmark/bert-base-cased/article_bias_prediction,2/10 (20.0 %),2/10 (20.0 %),7/10 (70.0 %),4/10 (40.0 %),2/10 (20.0 %),3/10 (30.0 %),1/10 (10.0 %)
dataset_benchmark/bert-base-cased/commoncrawl_news_articles,6/10 (60.0 %),2/10 (20.0 %),6/10 (60.0 %),4/10 (40.0 %),4/10 (40.0 %),3/10 (30.0 %),1/10 (10.0 %)
dataset_benchmark/bert-base-cased/gpt4_political_bias,6/10 (60.0 %),2/10 (20.0 %),5/10 (50.0 %),4/10 (40.0 %),4/10 (40.0 %),3/10 (30.0 %),3/10 (30.0 %)
